In [1]:
# 1. Install required packages
# Run this once in your environment
# pip install kaggle pandas sqlite3

import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
# %pip install seaborn

In [2]:
print(os.getcwd())

/Users/mac2025/Desktop/Continuous Education/Scaler/Sessions/Portfolio_projects/BeginnerSQLProject


In [5]:
# 3. Load CSV into pandas
df = pd.read_csv('/Users/mac2025/Desktop/Continuous Education/Scaler/Sessions/Portfolio_projects/BeginnerSQLProject/Sales Data.csv')

In [6]:
conn = sqlite3.connect('SalesData.db')
df.to_sql('SalesData', conn, index=False, if_exists='replace')

185950

In [18]:
def query_and_print(sql):
    result = pd.read_sql(sql, conn)
    print(result, '\n')

analysis_queries = {
    'Segment customers by total spending': '''
        SELECT 
          "Purchase Address" AS customer_identifier,
          SUM("Sales") AS total_spent,
          CASE
            WHEN SUM("Sales") > 1000 THEN 'High-value'
            WHEN SUM("Sales") BETWEEN 500 AND 1000 THEN 'Medium-value'
            ELSE 'Low-value'
          END AS spending_segment
        FROM SalesData
        GROUP BY "Purchase Address"
        ORDER BY total_spent DESC;
    ''',
    'Segment customers by purchase frequency': '''
        SELECT 
          "Purchase Address" AS customer_identifier,
          COUNT("Order ID") AS purchase_count,
          CASE
            WHEN COUNT("Order ID") > 20 THEN 'Frequent Buyer'
            WHEN COUNT("Order ID") BETWEEN 10 AND 20 THEN 'Moderate Buyer'
            ELSE 'Occasional Buyer'
          END AS frequency_segment
        FROM SalesData
        GROUP BY "Purchase Address"
        ORDER BY purchase_count DESC;
    ''',
    'Segment customers top product category': '''
        SELECT 
          "Purchase Address" AS customer_identifier,
          Product,
          COUNT("Order ID") AS purchases
        FROM SalesData
        GROUP BY "Purchase Address", Product
        ORDER BY "Purchase Address", purchases DESC;
    ''',
    'Segment customers by last purchase date': '''
        SELECT 
          "Purchase Address",
          MAX("Order Date") AS last_purchase_date,
          CASE
            WHEN MAX("Order Date") > DATE('now', '-30 day') THEN 'Recent'
            WHEN MAX("Order Date") BETWEEN DATE('now', '-90 day') AND DATE('now', '-31 day') THEN 'Dormant'
            ELSE 'Inactive'
          END AS recency_segment
        FROM SalesData
        GROUP BY "Purchase Address"
        ORDER BY last_purchase_date DESC;
    ''',
    'Segment customers RFM': '''
        WITH rfm AS (
          SELECT 
            "Purchase Address",
            MAX("Order Date") AS last_purchase_date,
            COUNT("Order ID") AS frequency,
            SUM("Sales") AS monetary
          FROM SalesData
          GROUP BY "Purchase Address"
        )
        SELECT
          "Purchase Address",
          CASE
            WHEN last_purchase_date > DATE('now', '-30 day') THEN 'Recent'
            ELSE 'Lapsed'
          END AS recency,
          CASE
            WHEN frequency > 20 THEN 'Frequent'
            WHEN frequency BETWEEN 10 AND 20 THEN 'Moderate'
            ELSE 'Rare'
          END AS frequency_segment,
          CASE
            WHEN monetary > 1000 THEN 'High Spender'
            WHEN monetary BETWEEN 500 AND 1000 THEN 'Medium Spender'
            ELSE 'Low Spender'
          END AS monetary_segment
        FROM rfm
        ORDER BY monetary DESC;
    ''',
    'Top 10 Customers by revenue and purchase frequency': '''
        WITH customer_metrics AS (
          SELECT
            "Purchase Address",
            SUM("Sales") AS total_revenue,
            COUNT("Order ID") AS purchase_count
          FROM SalesData
          GROUP BY "Purchase Address"
        ),
        ranked_customers AS (
          SELECT
            "Purchase Address",
            total_revenue,
            purchase_count,
            NTILE(10) OVER (ORDER BY total_revenue DESC) AS revenue_decile,
            NTILE(10) OVER (ORDER BY purchase_count DESC) AS frequency_decile
          FROM customer_metrics
        )
        SELECT
          "Purchase Address",
          total_revenue,
          purchase_count
        FROM ranked_customers
        WHERE revenue_decile = 1
          AND frequency_decile = 1
        ORDER BY total_revenue DESC, purchase_count DESC
        LIMIT 10;
    ''',
    'Cluster Customers': '''
        WITH customer_category_summary AS (
          SELECT
            "Purchase Address",
            Product,
            SUM("Sales") AS category_spent,
            COUNT("Order ID") AS category_purchases
          FROM SalesData
          GROUP BY "Purchase Address", Product
        ),
        customer_ltv AS (
          SELECT
            "Purchase Address",
            SUM("Sales") AS lifetime_value
          FROM SalesData
          GROUP BY "Purchase Address"
        ),
        category_counts AS (
          SELECT
            "Purchase Address",
            COUNT(DISTINCT Product) AS distinct_categories
          FROM SalesData
          GROUP BY "Purchase Address"
        ),
        customer_clusters AS (
          SELECT 
            c."Purchase Address",
            l.lifetime_value,
            cc.distinct_categories,
            CASE 
              WHEN l.lifetime_value > 1000 THEN 'High LTV'
              WHEN l.lifetime_value BETWEEN 500 AND 1000 THEN 'Medium LTV'
              ELSE 'Low LTV'
            END AS ltv_segment,
            CASE 
              WHEN cc.distinct_categories > 5 THEN 'Diverse Buyer'
              WHEN cc.distinct_categories BETWEEN 2 AND 5 THEN 'Moderate Buyer'
              ELSE 'Niche Buyer'
            END AS category_segment
          FROM customer_category_summary c
          JOIN customer_ltv l ON c."Purchase Address" = l."Purchase Address"
          JOIN category_counts cc ON c."Purchase Address" = cc."Purchase Address"
          GROUP BY c."Purchase Address", l.lifetime_value, cc.distinct_categories
        )
        SELECT *
        FROM customer_clusters;
    '''
}

In [19]:
for desc, sql in analysis_queries.items():
    print(f'-- {desc} --')
    query_and_print(sql)

conn.close()

-- Segment customers by total spending --
                         customer_identifier  total_spent spending_segment
0       668 Park St, San Francisco, CA 94016      4379.99       High-value
1              795 1st St, Atlanta, GA 30301      4100.00       High-value
2              391 1st St, Seattle, WA 98101      4100.00       High-value
3         10 1st St, San Francisco, CA 94016      4000.00       High-value
4       731 11th St, New York City, NY 10001      3919.88       High-value
...                                      ...          ...              ...
140782       1 Willow St, Portland, OR 97035         2.99        Low-value
140783   1 Lincoln St, Los Angeles, CA 90001         2.99        Low-value
140784    1 Lake St, New York City, NY 10001         2.99        Low-value
140785  1 Church St, New York City, NY 10001         2.99        Low-value
140786           1 4th St, Seattle, WA 98101         2.99        Low-value

[140787 rows x 3 columns] 

-- Segment customers by purch